In [1]:
from cng.utils import *

import ibis
from ibis import _

con = ibis.duckdb.connect(extensions = ["spatial", "h3"])
set_secrets(con)
duckdb_install_h3()


In [2]:
taxa = "Fungi"
gpkg = f"/home/jovyan/nvme/public-inat/iNaturalist_geomodel_{taxa}.gpkg"

In [3]:
con.read_geo(gpkg, "inat")

DatabaseTable: inat
  taxon_id          int32
  parent_taxon_id   int32
  name              string
  rank              string
  iconic_taxon_id   int32
  iconic_taxon_name string
  geomodel_version  string
  geom              geospatial:geometry

In [13]:
def geom_to_h3(con, 
               tbl, 
               cols = "taxon_id, parent_taxon_id, name, rank, iconic_taxon_id, iconic_taxon_name",
               zoom = "3"):
    
    con.sql(f'''
      WITH t2 AS (
        WITH t1 AS (
          SELECT {cols},  ST_Dump(geom) AS geom 
          FROM {tbl}
        ) 
        SELECT {cols},
              h3_polygon_wkt_to_cells_string(UNNEST(geom).geom, {zoom}) AS h{zoom}
        FROM t1
      )
      SELECT {cols}, UNNEST(h{zoom}) AS h{zoom} FROM t2
      ''')




In [ ]:
geom_to_h3(con, "inat").to_parquet(f"s3://public-inat/rangemaps/{taxa}.parquet")

In [12]:
con.read_parquet("https://minio.carlboettiger.info/public-inat/rangemaps/Fungi.parquet")

DatabaseTable: ibis_read_parquet_xd74iymourbq7haknk5rggwbyu
  taxon_id          int32
  parent_taxon_id   int32
  name              string
  rank              string
  iconic_taxon_id   int32
  iconic_taxon_name string
  h3                string